In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
import cv2
import sys
import tensorflow as tf
import keras
import time

from PIL import Image

from distutils.core import setup
from setuptools import find_packages

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense,BatchNormalization

import multiprocessing

# Installing BCI Connection

In [ ]:
#--------------- SETUP--------------#

with open("README.md", "r") as fh:
    long_description = fh.read()

setup(
  name = 'pyOpenBCI',
  packages = find_packages(),
  version = '0.13',
  license='MIT',
  description = 'A lib for controlling OpenBCI devices',
  long_description=long_description,
  long_description_content_type="text/markdown",
  author = 'OpenBCI, Inc.',
  author_email = 'contact@openbci.com',
  url = 'https://github.com/andreaortuno/pyOpenBCI',
  download_url = 'https://github.com/andreaortuno/pyOpenBCI/archive/0.13.tar.gz',
  keywords = ['device', 'control', 'eeg', 'emg', 'ekg', 'ads1299', 'openbci', 'ganglion', 'cyton', 'wifi'],
  install_requires=[
          'numpy',
          'pyserial',
          'bitstring',
          'xmltodict',
          'requests',
      ] + ["bluepy >= 1.2"] if sys.platform.startswith("linux") else [],
  classifiers=[
    'Development Status :: 3 - Alpha',
    'Intended Audience :: Developers',
    'Topic :: Software Development :: Build Tools',
    'License :: OSI Approved :: MIT License',
    'Programming Language :: Python :: 3',
    'Programming Language :: Python :: 2.7',
    'Programming Language :: Python :: 3.4',
    'Programming Language :: Python :: 3.6',
  ],
)

# https://github.com/openbci-archive/pyOpenBCI/blob/master/setup.py

In [ ]:
## Installation
#First, make sure you have the necessary dependencies.

pip install numpy pyserial bitstring xmltodict requests

pip install -i https://test.pypi.org/simple/ pyOpenBCI
#@@ -149,6 +156,8 @@ The Wifi Shield already outputs the data in Volts and the aux data in G.

### Example (Print Raw Data)
#To test this example, use `py Examples\print_raw_example.py` or `python Examples\print_raw_example.py`.

from pyOpenBCI import OpenBCICyton
#@@ -162,6 +171,9 @@ board.start_stream(print_raw)


### Example (Simple LSL Streamer)
#To run this example, use `py Examples\lsl_example.py` or `python Examples\lsl_example.py`.

from pyOpenBCI import OpenBCICyton

# https://github.com/openbci-archive/pyOpenBCI/commit/63cff4c1459434b89e8b96f3fd755b8a6673b438

In [ ]:
# START STREAM

def print_raw(sample):
    print(sample.channels_data)

board = OpenBCICyton()
#Set (daisy = True) to stream 16 ch 
board = OpenBCICyton(daisy = False)

board.start_stream(print_raw)

In [ ]:
# turn into dataframe and add timestamp column 'Time'

# Predicting Emotion from EEG data

In [ ]:
# Loading the csv data
eeg= pd.read_csv(r"C:\Users\Zoe Mercury\Desktop\Iron II\A.I\EEG emotions.csv" ) 

# Encoding the categories into numbers
# Neg= 0  Neu= 1 Pos= 2
le = preprocessing.LabelEncoder()
le.fit(eeg['label'])
eeg['labels']=le.transform(eeg['label'])
eeg2=eeg.drop('label',axis=1)

# Train Test Split
X=eeg2.drop('labels',axis=1)
y=eeg2['labels']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 1)


# Random Forrest Regression
forest = RandomForestRegressor(n_estimators=12, 
                               max_depth=5,
                               random_state=1) 
forest.fit(X_train, y_train)
print('train accuracy was',forest.score(X_train,y_train))

pred = forest.predict(X_test)
print('test accuracy was',forest.score(X_test,y_test))
comparison=pd.DataFrame({'observed':y_test, 'predicted':pred})
display(comparison)


# Save model
pickle.dump(forest, open('C:\\Users\\Zoe Mercury\\Desktop\\Iron II\\A.I\\forest.pickle', 'wb'))
# PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Zoe Mercury\\Desktop\\Iron II\\A.I' because I didn't add a filename+extension to the path

# Load model
forest_model = pickle.load(open('C:\\Users\\Zoe Mercury\\Desktop\\Iron II\\A.I\\forest.pickle','rb'))

# Model is receiving data from OpenBCI and predicting
# 1 brainwave input per second
# get BCI output and convert into dataframe
# figure out how to add a new row every instance and only predict for the new row
# then pred= forest.predict(df)

# Real-Time Emotion Recognition via Webcam

In [ ]:
#-----------Unpacking fer2013 image library----------#

def create_image(column):
    images=pd.read_csv('fer2013.csv')
    width = 48
    height = 48
    a=0
    b=0
    c=0
    d=0
    e=0
    f=0
    g=0
    img  = Image.new( mode = "RGB", size = (width, height) )
    img.putdata(pixels)
    if images['emotion']==0:
        img.save('angry_'+ str(a) +'.png')
        a +=1
    elif images['emotion']==1:
        img.save('disgust_'+ str(b) +'.png')
        b +=1
    elif images['emotion']==2:
        img.save('fear_'+ str(c) +'.png')
        c +=1
    elif images['emotion']==3:
        img.save('happy_'+ str(d) +'.png')
        d +=1
    elif images['emotion']==4:
        img.save('sad_'+ str(e) +'.png')
        e +=1
    elif images['emotion']==5:
        img.save('surprise_'+ str(f) +'.png')
        f +=1
    else:
        img.save('neutral_'+ str(g) +'.png')
        g +=1
        
images['pixels'].apply(create_image)


# 0: -4593 images- Angry
# 1: -547 images- Disgust
# 2: -5121 images- Fear
# 3: -8989 images- Happy
# 4: -6077 images- Sad
# 5: -4002 images- Surprise
# 6: -6198 images- Neutral


In [ ]:
#-------------------TRAINING THE MODEL----------------#
# -----------Sequential Neural Network----------------#

model = Sequential()
model.add(Conv2D(32,kernel_size=2, input_shape=(450, 450, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32,kernel_size=2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(127,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(270,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(127,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(270,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,kernel_size=2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) 
model.add(Dense(64)) 
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))
# how many output categories there are

model.compile(loss='sparse_categorical_crossentropy',
              # expects labelEncoded input
              # binary_crossentropy expects binary output
              # categorical_crossentropy expects one-hot-encoded input
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(
        np.array(X), np.array(y),
        epochs=5000,
        batch_size=10,
        validation_split=0.12,
        shuffle=True,
        workers=10,
        use_multiprocessing=True)


model.save('model.h5')

In [ ]:
#-----------------RUNNING THE MODEL----------------#

def video_detector():
    resultlist=[]
    timestamps=[]
    model = tf.keras.models.load_model(r"C:\\Users\\Zoe Mercury\\Desktop\\Iron II\\A.I\\Untitled Folder\\model_small.h5")  
    face_cascade = cv2.CascadeClassifier("C:\\Users\\Zoe Mercury\\Desktop\\Iron II\\A.I\\Untitled Folder\\cropped\\haarcascade_frontalface_default.xml") 
    cap = cv2.VideoCapture(0)

    emotions = ('sad','happy','angry','excited')

    while True:
        ret, img = cap.read()
        frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faces = face_cascade.detectMultiScale(frame, 1.3, 5)

        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray=frame[y:y+w,x:x+h]
            roi_gray=cv2.resize(frame,(200,200))  
            img_pixels = np.array(roi_gray)  / 255
            img_pixels = np.expand_dims(img_pixels, axis = 0)  

            predictions = model.predict(img_pixels)  
            print(predictions)
            max_index = np.argmax(predictions[0])  
            print(max_index)
            resultlist.append(max_index)
            timestamps.append((datetime.now()))
            d= {'Video Prediction': resultlist,'Time':timestamps}
            df = pd.DataFrame(d, columns=['Video Prediction','Time'])

            predicted_emotion = emotions[max_index] + ' '+ str(int(predictions[0][max_index]*100)) +'%'

            cv2.putText(img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)  


        cv2.imshow('img',img)

        if cv2.waitKey(10) == ord('q'):
            break  

    cap.release()
    cv2.destroyAllWindows()
    return df

# Parallelization of Processes

In [ ]:
start = time.perf_counter()
p1 = multiprocessing.Process(target = video_detector, args = [2])
p2 = multiprocessing.Process(target = eeg_detector, args = [2])

p1.start()
p2.start()

p1.join()
p2.join()

finish = time.perf_counter()
print('finished in ' + str(finish-start)+' seconds')

# Pokerface or not?

In [ ]:
#--------------COMPARE RESULTS----------------

df.merge(comparison,on='Time')

if df['EEG Prediction']==df['Video Prediction']:
    print('You are honest!')
else:
    print('Mismatch of Emotion and Facial Expression!')
    
    